In [2]:
import requests
import re 
import os 
import time
import traceback 
from bs4 import BeautifulSoup 
from lxml import etree
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from random import randint

In [3]:
city_url = 'https://raw.githubusercontent.com/ucsb-dreamlab/dianpingBusinessScrape/master/City_URL.csv'
city = pd.read_csv(city_url, index_col= 0)
city.head(10)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
5,安顺,www.dianping.com/anshun/ch75/g34311
6,阿里,www.dianping.com/ali/ch75/g34311
7,安康,www.dianping.com/ankang/ch75/g34311
8,安国市,www.dianping.com/anguo/ch75/g34311
9,安新县,www.dianping.com/anxin/ch75/g34311


Filter this dataset by city levels. Now it has 411 results, and most cities in this filtered dataset are tier 1-3 city. 

In [4]:
filtered_city = city[-(
                city['city_name'].str.endswith('市')| 
                city['city_name'].str.endswith('县')|
                city['city_name'].str.endswith('镇')| 
                city['city_name'].str.endswith('旗')|
                city['city_name'].str.endswith('村')| 
                city['city_name'].str.endswith('区'))].reset_index(drop=True)
display(filtered_city)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
...,...,...
406,诸暨,www.dianping.com/zhuji/ch75/g34311
407,章丘,www.dianping.com/zhangqiu/ch75/g34311
408,周庄,www.dianping.com/zhouzhuang/ch75/g34311
409,朱家尖,www.dianping.com/zhujiajian/ch75/g34311


In [5]:
def getHtmlContent(url): 
 UA = UserAgent(verify_ssl = False)   
 headers = {
    "User-Agent": UA.random
    }

 try:
   s = requests.Session()
   r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
      #r = s.get(url, headers = headers, cookies= s.cookies)
   r.raise_for_status()
   if(r.encoding != "UTF-8"):
      r.encoding = r.apparent_encoding
   return r.text
 except:
   return "Request Error"

In [6]:
URL =  "http://"+ filtered_city['URL'][1] # add "https" to avoid schema error 
URL

'http://www.dianping.com/anshan/ch75/g34311'

In [7]:
HTML = getHtmlContent(URL)

In [8]:
soup = BeautifulSoup(HTML, "html.parser")
print(soup)

<html>
<head>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="icon" type="image/x-icon"/>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="shortcut icon" type="image/x-icon"/>
<title>鞍山绘本馆机构_课程_价格_排名-大众点评网</title>
<meta content="鞍山绘本馆,鞍山绘本馆机构,鞍山绘本馆课程,鞍山绘本馆价格,鞍山绘本馆机构排名,大众点评网" name="Keywords"/>
<meta content="大众点评网学习培训频道为您找到最新最全的鞍山绘本馆机构。点击查看更多关于鞍山绘本馆机构课程、价格、评价、排名、电话、地址等信息。" name="Description"/>
<meta content="format=html5; url=https://m.dianping.com/anshan/ch75/g34311" http-equiv="mobile-agent"/>
<link href="https://m.dianping.com/anshan/ch75/g34311" media="only screen and (max-width: 640px)" rel="alternate"/>
<meta charset="utf-8">
<script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '58',                   
            'cityChName': '鞍山',                 
            'cityEnName': 'anshan',               
            'pageType': 'search',                  

In [9]:
SubURL = []
LibName = []
for div in soup.find_all(class_ = "tit"):
    for a in div.find_all('a', href = True):
        SubURL.append(a.get('href'))
        LibName.append(a.getText().strip())

//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [10]:
city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
city_lib

,Library_Name,URL
0,盛文绘本馆,https://www.dianping.com/shop/G7NqN7b6XuDR1Av9
1,花瓣绘本馆,https://www.dianping.com/shop/jKWN2u1yfFov84SD
2,啄木鸟,https://www.dianping.com/shop/l7uvWW0qRkpJajex
3,不点的小书库(鞍山大悦城店),https://www.dianping.com/shop/G64Zmx3FjlgEZzJp
4,兰可琪绘本美学生活馆,https://www.dianping.com/shop/H5t9xip4GPLuIlNt
5,啄木鸟,https://www.dianping.com/shop/jVQ42SQoGAhH3HC4
6,布朗熊亲子绘本馆,https://www.dianping.com/shop/H2hW0bGhmUci0kDT


In [11]:
subpage_soup = []
for i in range(0,len(city_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = city_lib['URL'][i]), "html.parser"))
    time.sleep(randint(5,10))

In [12]:
subpage_soup[1]

<!DOCTYPE html>

<html>
<head>
<link href="http://www.dianping.com/shop/jKWN2u1yfFov84SD" rel="canonical"/>
<script type="text/javascript">(function(){var g=["iPhone","Android","Windows Phone"],b="http://m.dianping.com",e=window.location.pathname+window.location.search,f=false;for(var d=0,a;a=g[d];d++){if(navigator.userAgent.indexOf(a)!=-1){f=true;break}}if(document.referrer.indexOf("baidu.com")>0){if(window.location.search==""){e=e+"?msource=baidusearch"}else{e=e+"&msource=baidusearch"}}else if(document.referrer.indexOf("sogou.com")>0){if(window.location.search==""){e=e+"?msource=sogousearch"}else{e=e+"&msource=sogousearch"}}function h(){location.href=b+e}function c(){if(document.referrer.indexOf(b)==0){document.cookie="vmod=pc;expires=0;"}return document.cookie.match(/vmod=pc/)}if(f){if(!c()){h()}}})();</script>
<title>花瓣绘本馆课程_价格_简介_怎么样-鞍山学习培训-大众点评网</title>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="pc" name="applicable-device"/>
<meta content="花瓣绘本馆,花瓣绘本馆课

In [13]:
AddressList = []
HourList = []
PhoneNumList = []


In [14]:
for soup in subpage_soup:
    for div in soup.find_all(class_ = "address"):
            AddressList.append(div.find_all(text=True, recursive=False)[1].strip())

In [15]:
AddressList

['二一九路26号',
 '自由街劳动大厦北50米',
 '景子街华润万家超市负一层',
 '胜利北路302号鞍山大悦城F4层',
 '工人街30栋向阳小学东30米',
 '新昌大街西50米',
 '新东三街与新东二街交叉口东北260米']

In [16]:

for soup in subpage_soup:
    hour_tag = soup.find(class_ = "mod shop-info")
    if hour_tag is not None:
       for li in hour_tag.find_all('li'):
          str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
          HourList.append(str) 
    else:
          HourList.append(np.NaN)

In [17]:
     #for li in div.find_all('li'):
         #   try:
           #     HourList.append(li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")) 
            #except IndexError:
              #  HourList.append(np.NaN)
                   

In [18]:
HourList

[nan,
 '周二至周日 10:00-13:00 13:30-18:30',
 '周一至周日 09:00-21:00',
 nan,
 '周一至周日 10:00-22:00',
 '周一至周日 09:00-21:00',
 '周一至周日 09:00-18:00']

In [19]:
for soup in subpage_soup: 
    phone_tag = soup.find(class_ = "phone")
    if phone_tag is not None:
        phonenum = phone_tag.find('span')
        PhoneNumList.append(phonenum['data-phone'])
    else: 
        PhoneNumList.append(np.NaN)

In [20]:
PhoneNumList

[nan,
 '15998001940',
 nan,
 '13609819477',
 '15604229606',
 '15941245428',
 '13566634575']

In [21]:
city_lib['Address'] = AddressList
city_lib['Business_Hour'] = HourList
city_lib['Phone_Number'] = PhoneNumList
city_lib

,Library_Name,URL,Address,Business_Hour,Phone_Number
0,盛文绘本馆,https://www.dianping.com/shop/G7NqN7b6XuDR1Av9,二一九路26号,NaN,NaN
1,花瓣绘本馆,https://www.dianping.com/shop/jKWN2u1yfFov84SD,自由街劳动大厦北50米,周二至周日 10:00-13:00 13:30-18:30,15998001940
2,啄木鸟,https://www.dianping.com/shop/l7uvWW0qRkpJajex,景子街华润万家超市负一层,周一至周日 09:00-21:00,NaN
3,不点的小书库(鞍山大悦城店),https://www.dianping.com/shop/G64Zmx3FjlgEZzJp,胜利北路302号鞍山大悦城F4层,NaN,13609819477
4,兰可琪绘本美学生活馆,https://www.dianping.com/shop/H5t9xip4GPLuIlNt,工人街30栋向阳小学东30米,周一至周日 10:00-22:00,15604229606
5,啄木鸟,https://www.dianping.com/shop/jVQ42SQoGAhH3HC4,新昌大街西50米,周一至周日 09:00-21:00,15941245428
6,布朗熊亲子绘本馆,https://www.dianping.com/shop/H2hW0bGhmUci0kDT,新东三街与新东二街交叉口东北260米,周一至周日 09:00-18:00,13566634575


In [22]:
#Summarized the above code into a function
def parse_info_citylib(URL):
    subpage_soup = []
    for i in range(0,6): #Need to change this part
        subpage_soup.append(BeautifulSoup(getHtmlContent(url = URL), "html.parser"))
        time.sleep(3)

    AddressList = []
    HourList = []
    PhoneNumList = []

    for soup in subpage_soup:
        for div in soup.find_all(class_ = "address"):
            AddressList.append(div.find_all(text=True, recursive=False)[1].strip())
    
    for soup in subpage_soup:    
        hour_tag = soup.find(class_ = "mod shop-info")
        if hour_tag is not None:
            for li in hour_tag.find_all('li'):
                str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                HourList.append(str) 
            else:
                HourList.append(np.NaN)
    
    for soup in subpage_soup:
        phone_tag = soup.find(class_ = "phone")
        if phone_tag is not None:
            phonenum = phone_tag.find('span')
            PhoneNumList.append(phonenum['data-phone'])
        else: 
            PhoneNumList.append(np.NaN)
            
    return AddressList, HourList, PhoneNumList

In [23]:
Second_URL="http://"+ filtered_city['URL'][2]
lib_info = parse_info_citylib(Second_URL)